In [1]:
print("Starting the parsing program...")

Starting the parsing program...


In [4]:
%pip install requests psycopg2 python-dotenv pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.3 MB 11.7 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.3 MB 11.9 MB/s eta 0:00:01
   ------------------------- -------------- 7.3/11.3 MB 11.9 MB/s eta 0:00:01
   ----------------------------------- ---- 10.0/11.3 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 11.2 MB/s  0:00:01
   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   ----- ---------------------------------- 1.8/13.1 MB 12.6 MB/s eta 0:00:01
   ------------ --------------------------- 4.2/13.1 MB 12.0 MB/s eta 0:00:01
   -------------------- ------------------- 6.8/13.1 MB 12.0 MB/s eta 0:00:01
   --------------------------- ------------ 8.9/13.1 MB 11.5 MB/s eta 0:00:01
   -----------------------

In [5]:
import requests
import json
import psycopg2
import os
import dotenv
from datetime import datetime
import pandas as pd

In [20]:
BASE_URL = "https://v3.football.api-sports.io"

url_leagues = f"{BASE_URL}/leagues"
url_fixtures = f"{BASE_URL}/fixtures"

headers = {
  'x-rapidapi-key': '316dede52d52efb30d6dcc7cfb2e936b',
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

In [ ]:
payload={}

response = requests.request("GET", url_leagues, headers=headers, data=payload)

In [ ]:
with open("leagues_response.json", "w", encoding="utf-8") as f:
    json.dump(response.json(), f, ensure_ascii=False, indent=4)

In [ ]:
with open("leagues_response.json", "r", encoding="utf-8") as f:
    data_league = json.load(f)

In [ ]:
for league in data_league['response']:
    if league['country']['name'] in ['England']:
        print(f"League id = {league['league']['id']}: {league['league']['name']}, Country: {league['country']['name']}")

League id = 39: Premier League, Country: England
League id = 40: Championship, Country: England
League id = 46: EFL Trophy, Country: England
League id = 45: FA Cup, Country: England
League id = 47: FA Trophy, Country: England
League id = 48: League Cup, Country: England
League id = 43: National League, Country: England
League id = 41: League One, Country: England
League id = 42: League Two, Country: England
League id = 528: Community Shield, Country: England
League id = 44: FA WSL, Country: England
League id = 49: National League - Play-offs, Country: England
League id = 698: FA Women's Cup, Country: England
League id = 50: National League - North, Country: England
League id = 51: National League - South, Country: England
League id = 52: Non League Div One - Isthmian North, Country: England
League id = 53: Non League Div One - Isthmian South Central, Country: England
League id = 54: Non League Div One - Northern West, Country: England
League id = 55: Non League Div One - Northern Midla

In [ ]:
def get_all_fixtures(league_id: int, season: int):
    page = 1
    fixtures = []

    while True:
        params = {
            "league": league_id,
            "season": season
        }
        response = requests.get(url_fixtures, headers=headers, params=params)
        data = response.json()

        if "response" not in data or not data["response"]:
            break

        fixtures.extend(data["response"])
        
        # Проверяем, есть ли следующая страница
        paging = data.get("paging", {})
        if paging.get("current") == paging.get("total"):
            break

        page += 1

    return fixtures

In [27]:
params = {
    "league": 39,
    "season": 2023
}

response = requests.get(url_fixtures, headers=headers, params=params)

In [29]:
with open("fixtures.json", "w", encoding="utf-8") as f:
    json.dump(response.json(), f, ensure_ascii=False, indent=2)

In [30]:
with open("fixtures.json", "r", encoding="utf-8") as f:
    fixtures = json.load(f)

In [33]:
for fixture in fixtures['response']:
    print(f"Fixture ID: {fixture['fixture']['id']}, Date: {fixture['fixture']['date']}, Teams: {fixture['teams']['home']['name']} vs {fixture['teams']['away']['name']}, Score: {fixture['goals']['home']} - {fixture['goals']['away']}")

Fixture ID: 1035037, Date: 2023-08-11T19:00:00+00:00, Teams: Burnley vs Manchester City, Score: 0 - 3
Fixture ID: 1035038, Date: 2023-08-12T11:30:00+00:00, Teams: Arsenal vs Nottingham Forest, Score: 2 - 1
Fixture ID: 1035039, Date: 2023-08-12T14:00:00+00:00, Teams: Bournemouth vs West Ham, Score: 1 - 1
Fixture ID: 1035041, Date: 2023-08-12T14:00:00+00:00, Teams: Everton vs Fulham, Score: 0 - 1
Fixture ID: 1035040, Date: 2023-08-12T14:00:00+00:00, Teams: Brighton vs Luton, Score: 4 - 1
Fixture ID: 1035042, Date: 2023-08-12T14:00:00+00:00, Teams: Sheffield Utd vs Crystal Palace, Score: 0 - 1
Fixture ID: 1035043, Date: 2023-08-12T16:30:00+00:00, Teams: Newcastle vs Aston Villa, Score: 5 - 1
Fixture ID: 1035044, Date: 2023-08-13T13:00:00+00:00, Teams: Brentford vs Tottenham, Score: 2 - 2
Fixture ID: 1035045, Date: 2023-08-13T15:30:00+00:00, Teams: Chelsea vs Liverpool, Score: 1 - 1
Fixture ID: 1035046, Date: 2023-08-14T19:00:00+00:00, Teams: Manchester United vs Wolves, Score: 1 - 0
Fixtu

In [ ]:
# Пример: Английская Премьер-лига (league_id=39), сезон 2023
fixtures = get_all_fixtures(league_id=39, season=2023)

# Сохраняем все raw-данные в JSON
with open("fixtures.json", "w", encoding="utf-8") as f:
    json.dump(fixtures, f, ensure_ascii=False, indent=2)

In [ ]:

# Переводим в DataFrame
rows = []
for f in fixtures:
    rows.append({
        "fixture_id": f["fixture"]["id"],
        "date": f["fixture"]["date"],
        "home_team": f["teams"]["home"]["name"],
        "away_team": f["teams"]["away"]["name"],
        "home_score": f["score"]["fulltime"]["home"],
        "away_score": f["score"]["fulltime"]["away"]
    })

df = pd.DataFrame(rows)

print(df.head())
print(f"Всего матчей: {len(df)}")


In [6]:
def get_conn_to_pg():
    from dotenv import load_dotenv
    load_dotenv()
    return psycopg2.connect(
        database=os.environ['PG_DB'],
        user=os.environ['PG_USR'],
        password=os.environ['PG_PWD'],
        host=os.environ['PG_HOST'],
        port=os.environ['PG_PORT']
    )

In [ ]:
# Load data from JSON file
def load_leagues_to_db(file_name):
    with open(file_name, "r", encoding="utf-8") as f:
        data_league = json.load(f)

    # Connect to PostgreSQL
    conn = get_conn_to_pg()
    cur = conn.cursor()

    countries = set()
    for league in data_league['response']:
        country = league['country']['name']
        code = league['country']['code']
        flag = league['country']['flag']
        countries.add((country, code, flag))

    countries_with_id = [
        (idx, name, code, flag)
        for idx, (name, code, flag) in enumerate(sorted(list(countries), key=lambda x: x[0]), start=1)
    ]
    created_at = datetime.now().isoformat()
    updated_at = datetime.now().isoformat()

    for country in countries_with_id:
        id = country[0]
        name = country[1]
        code = country[2]
        flag_image = country[3]
        # print(code)
        cur.execute(
            "INSERT INTO public.football_app_country (id, name, code, flag_image, created_at, updated_at) VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT (id) DO NOTHING",
            (id, name, code, flag_image, created_at, updated_at)
        )

    for id, league in enumerate(data_league['response']):
        external_id = league['league']['id']
        league_name = league['league']['name']
        country = league['country']['name']
        logo_image = league['league']['logo']
        type = league['league']['type']
        country_id = next((c[0] for c in countries_with_id if c[1] == country), None)
        is_active = False
        print(id, league_name)
        cur.execute(
            "INSERT INTO public.football_app_league  (id, name, logo_image, is_active, country_id, external_id, type, created_at, updated_at) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (id) DO NOTHING",
            (id, league_name, logo_image, is_active, country_id, external_id, type, created_at, updated_at)
        )

        for season in league['seasons']:            
            season_start_year = season['year']
            season_current = season['current']
            season_start = season['start']
            season_end = season['end']
            season_end_year = datetime.strptime(season['end'], "%Y-%m-%d").year if season['end'] else season_start
            season_name = f"{league_name} {season_start_year}/{str(season_end_year)[-2:]}"
            season_coverage_fixtures_events = season['coverage']['fixtures']['events']
            season_coverage_fixtures_lineups = season['coverage']['fixtures']['lineups']
            season_coverage_fixtures_statistics_fixtures = season['coverage']['fixtures']['statistics_fixtures']
            season_coverage_fixtures_statistics_players = season['coverage']['fixtures']['statistics_players']
            season_coverage_standings = season['coverage']['standings']
            season_coverage_players = season['coverage']['players']
            season_coverage_top_scorers = season['coverage']['top_scorers']
            season_coverage_top_assists = season['coverage']['top_assists']
            season_coverage_top_cards = season['coverage']['top_cards']
            season_coverage_injuries = season['coverage']['injuries']
            season_coverage_predictions = season['coverage']['predictions']
            season_coverage_odds = season['coverage']['odds']
            season_is_active = True
            cur.execute(
                "INSERT INTO public.football_app_season ("
                "name, start_year, end_year, start_date, end_date, is_current, is_active," 
                "created_at, updated_at, coverage_fixtures_events, coverage_fixtures_lineups," 
                "coverage_fixtures_statistics_fixtures, coverage_fixtures_statistics_players," 
                "coverage_injuries, coverage_odds, coverage_players, coverage_predictions," 
                "coverage_standings, coverage_top_assists, coverage_top_cards, coverage_top_scorers, league_id)" 
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"  
                "ON CONFLICT (id) DO NOTHING",
            (season_name, season_start_year, season_end_year, season_start, season_end, 
             season_current, season_is_active, created_at, updated_at,
             season_coverage_fixtures_events,
             season_coverage_fixtures_lineups,
             season_coverage_fixtures_statistics_fixtures,
             season_coverage_fixtures_statistics_players,
             season_coverage_injuries,
             season_coverage_odds,
             season_coverage_players,
             season_coverage_predictions,
             season_coverage_standings, 
             season_coverage_top_assists,
             season_coverage_top_cards,
             season_coverage_top_scorers,
             id
             )
        )

    conn.commit()
    cur.close()
    conn.close()

In [8]:
file_name = "leagues_response.json"
load_leagues_to_db(file_name)

UnboundLocalError: cannot access local variable 'league_name' where it is not associated with a value

In [6]:
with open("leagues_response.json", "r", encoding="utf-8") as f:
    data_league = json.load(f)

In [ ]:
countries = set()
for league in data_league['response']:
    country = league['country']['name']
    code = league['country']['code']
    flag = league['country']['flag']
    countries.add((country, code, flag))

countries_with_id = [
    (idx, name, code, flag)
    for idx, (name, code, flag) in enumerate(sorted(list(countries), key=lambda x: x[0]), start=1)
]

In [ ]:
from collections import Counter

counts = Counter(c[2] for c in countries_with_id)
duplicates = [c for c in countries_with_id if counts[c[2]] > 1]
print(duplicates)


[(38, 'Crimea', 'UA', 'https://media.api-sports.io/flags/ua.svg'), (161, 'Ukraine', 'UA', 'https://media.api-sports.io/flags/ua.svg')]


In [ ]:
for id, league in enumerate(data_league['response']):
   if league['country']['name'] in ['England'] and league['league']['id'] in [39]:
      print(f"League id = {league['league']['id']}: {league['league']['name']}, Country: {league['country']['name']}")
      for season in league['seasons']:
         season_start_year = season['year']
         season_current = season['current']
         season_start = season['start']
         season_end = season['end']
         season_end_year = datetime.strptime(season['end'], "%Y-%m-%d").year
         season_coverage_fixtures_events = season['coverage']['fixtures']['events']
         season_coverage_fixtures_lineups = season['coverage']['fixtures']['lineups']
         season_coverage_fixtures_statistics = season['coverage']['fixtures']['statistics']
         season_coverage_fixtures_players_statistics = season['coverage']['fixtures']['players_statistics']
         season_coverage_standings = season['coverage']['standings']
         season_coverage_players = season['coverage']['players']
         season_coverage_top_scorers = season['coverage']['top_scorers']
         season_coverage_top_assists = season['coverage']['top_assists']
         season_coverage_top_cards = season['coverage']['top_cards']
         season_coverage_injuries = season['coverage']['injuries']
         season_coverage_predictions = season['coverage']['predictions']
         season_coverage_odds = season['coverage']['odds']
         season_is_active = True

         # print(f"  Season: {season['year']}, Current: {season['current']}, Start: {season['start']}, End: {season['end']}")

League id = 39: Premier League, Country: England
  Season: 2010, Current: False, Start: 2010-08-14, End: 2011-05-17
  Season: 2011, Current: False, Start: 2011-08-13, End: 2012-05-13
  Season: 2012, Current: False, Start: 2012-08-18, End: 2013-05-19
  Season: 2013, Current: False, Start: 2013-08-17, End: 2014-05-11
  Season: 2014, Current: False, Start: 2014-08-16, End: 2015-05-24
  Season: 2015, Current: False, Start: 2015-08-08, End: 2016-05-17
  Season: 2016, Current: False, Start: 2016-08-13, End: 2017-05-21
  Season: 2017, Current: False, Start: 2017-08-11, End: 2018-05-13
  Season: 2018, Current: False, Start: 2018-08-10, End: 2019-05-12
  Season: 2019, Current: False, Start: 2019-08-09, End: 2020-07-26
  Season: 2020, Current: False, Start: 2020-09-12, End: 2021-05-23
  Season: 2021, Current: False, Start: 2021-08-13, End: 2022-05-22
  Season: 2022, Current: False, Start: 2022-08-05, End: 2023-05-28
  Season: 2023, Current: False, Start: 2023-08-11, End: 2024-05-19
  Season: 202